In [343]:
# Import python modules

import pandas as pd
import glob
import datetime 
import requests
import os
import json
from pandas.io.json import json_normalize

from dotenv import load_dotenv
load_dotenv()

import coinbasepro as cbp
from coinbase.wallet.client import Client

%matplotlib inline

In [344]:
# Read API key from .env file

cb_public_key = os.getenv("CB_PUBLIC_KEY")
cb_secret_key = os.getenv("CB_SECRET_KEY")
client = Client(cb_public_key, cb_secret_key, api_version='YYYY-MM-DD')

In [345]:
# Create a function to pull one year data from CoinBase PRO API


def datapull(startDay, endDay, myCrypto):
    
    params = {"start" : startDay, "end" : endDay, "granularity" : "86400"}
    
    url = (f" https://api.pro.coinbase.com/products/{myCrypto}-USD/candles")

    requests.get(url)
    response_data = requests.get(url, params=params)
    response_content = response_data
    print(response_content)
    data = response_data.json()
        

    data_df = pd.DataFrame(data)
    data_df.columns = ["Date", "Low", "High", "Open", "Close", "Volume"]
    data_df["Date"]= pd.to_datetime(data_df['Date'],unit='s')
    
    return data_df

# Select crypto ticker to pull data for (this only needs to be changed to get final data output. If need to pull data for different timeframe change dates below). Because CoinbasePro allows only 300 datapoints in one pull, multiple pulls with data adjustment may be needed.

crypto ="REP"

# Adjust dates for the timeframe you need data for.

batch1 = datapull("2020-01-05T12:00:00","2020-10-30T12:00:00",crypto)
batch2 = datapull("2019-10-05T12:00:00","2020-01-05T12:00:00",crypto)

batch_combined_df = pd.concat([batch1,batch2], axis="rows", join="inner")

# Check for duplicate rows

batch_combined_df.drop_duplicates()

# Select one year data

batch_combined_df.insert(0, "Crypto", f"{crypto}")

batch_combined_df = batch_combined_df.iloc[0:365]

# Print output to csv file (enable code if need to print to .csv)

#batch_combined_df.to_csv(f"output_batch_combined_{crypto}.csv")

# Print crypto data in pandas dataframe format

batch_combined_df.head()


<Response [200]>
<Response [200]>


,Crypto,Date,Low,High,Open,Close,Volume
0,REP,2020-10-30,12.40,14.29,13.39,13.26,32709.218589
1,REP,2020-10-29,12.30,14.00,12.30,13.46,44082.969028
2,REP,2020-10-28,12.02,13.16,13.05,12.30,22020.868339
3,REP,2020-10-27,12.77,13.37,13.00,13.05,14239.214067
4,REP,2020-10-26,12.93,13.42,13.32,12.99,12894.203317


In [351]:
# Read multiple crypto data .csv files and concat into a dataframe

crypto_data = "../data"
csvfiles = glob.glob(os.path.join(crypto_data, "*csv"))

# loop through .csv files and read them 

crypto_df =[]

for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    crypto_df.append(df)

crypto_df

# Concat individual dfs into a comboned df and set date as index 

all_cryto_data_df = pd.concat(crypto_df, join="inner", axis="columns")
all_cryto_data_df.reset_index(drop=True,inplace=True)
all_cryto_data_df.drop("Unnamed: 0", axis=1, inplace=True)

all_cryto_data_df.head()


,Crypto,Date,Low,High,Open,Close,Volume,Crypto,Date,Low,...,Open,Close,Volume,Crypto,Date,Low,High,Open,Close,Volume
0,EOS,2020-10-30,2.402,2.649,2.640,2.510,550128.1,XLM,2020-10-30,0.074457,...,13.39,13.26,32709.218589,XRP,2020-10-30,0.2310,0.2442,0.2427,0.2395,4.749147e+07
1,EOS,2020-10-29,2.605,2.677,2.655,2.638,258459.5,XLM,2020-10-29,0.075419,...,12.30,13.46,44082.969028,XRP,2020-10-29,0.2400,0.2473,0.2457,0.2425,2.960889e+07
2,EOS,2020-10-28,2.621,2.755,2.673,2.654,401503.3,XLM,2020-10-28,0.077413,...,13.05,12.30,22020.868339,XRP,2020-10-28,0.2438,0.2569,0.2528,0.2457,4.575991e+07
3,EOS,2020-10-27,2.628,2.699,2.631,2.675,266275.7,XLM,2020-10-27,0.081034,...,13.00,13.05,14239.214067,XRP,2020-10-27,0.2479,0.2543,0.2485,0.2527,3.174011e+07
4,EOS,2020-10-26,2.597,2.751,2.714,2.634,347685.1,XLM,2020-10-26,0.079856,...,13.32,12.99,12894.203317,XRP,2020-10-26,0.2436,0.2593,0.2535,0.2486,4.414561e+07
